<a href="https://colab.research.google.com/github/nalivaikaanastasiya-dev/Agentic-RAG-Streamlit-App/blob/main/agent_rag_pipeline.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# agentic_rag_pipeline.py
import os
import logging
from dotenv import load_dotenv

from langchain.chat_models import ChatOpenAI
from langchain.chains import RetrievalQA
from langchain.agents import initialize_agent, Tool
from langchain.vectorstores import Chroma
from langchain.embeddings.openai import OpenAIEmbeddings
from langchain.tools import BaseTool
from langchain.docstore.document import Document

# -----------------------------
# 1️⃣ Загрузка переменных окружения
# -----------------------------
load_dotenv()
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
TAVILY_API_KEY = os.getenv("TAVILY_API_KEY")
CHROMA_DB_PATH = os.getenv("CHROMA_DB_PATH", "./chroma_db")
LANGSMITH_API_KEY = os.getenv("LANGSMITH_API_KEY")

# Настройка логирования
logging.basicConfig(level=logging.INFO)
logger = logging.getLogger("AgenticRAG")

# -----------------------------
# 2️⃣ Векторное хранилище
# -----------------------------
def create_vectordb(documents):
    """Создает векторное хранилище из списка документов"""
    try:
        embeddings = OpenAIEmbeddings(openai_api_key=GROQ_API_KEY)  # Можно заменить на Groq embeddings, когда доступно
        vectordb = Chroma.from_documents(documents, embeddings, persist_directory=CHROMA_DB_PATH)
        vectordb.persist()
        logger.info("Векторное хранилище создано")
        return vectordb
    except Exception as e:
        logger.error(f"Ошибка создания векторного хранилища: {e}")
        raise e

# -----------------------------
# 3️⃣ Инструмент Tavily Search
# -----------------------------
class TavilySearchTool(BaseTool):
    """Интерфейс для поиска через Tavily"""
    name = "TavilySearch"
    description = "Ищет информацию в интернете через Tavily"

    def _call(self, query: str) -> str:
        try:
            logger.info(f"Tavily search query: {query}")
            # TODO: Подключить реальный Tavily API
            return f"Результат поиска Tavily для запроса: {query}"
        except Exception as e:
            logger.error(f"Tavily search error: {e}")
            return f"Ошибка поиска: {str(e)}"

# -----------------------------
# 4️⃣ Создание агента
# -----------------------------
def create_agent(vectordb):
    """Создает агент с инструментами RAG и web search"""
    llm = ChatOpenAI(openai_api_key=GROQ_API_KEY, temperature=0)

    # RAG retrieval chain
    retriever = vectordb.as_retriever()
    retrieval_chain = RetrievalQA.from_chain_type(llm=llm, chain_type="stuff", retriever=retriever)

    # Инструменты агента
    tools = [
        Tool(name="RAG", func=retrieval_chain.run, description="Поиск по базе знаний"),
        TavilySearchTool()
    ]

    # Инициализация агента с инструментами
    agent = initialize_agent(tools, llm, agent="zero-shot-react-description", verbose=True)
    return agent

# -----------------------------
# 5️⃣ Python API для Streamlit
# -----------------------------
class AgenticRAG:
    def __init__(self, documents):
        self.vectordb = create_vectordb(documents)
        self.agent = create_agent(self.vectordb)

    def get_answer(self, query: str) -> str:
        """Возвращает ответ агента на запрос пользователя"""
        try:
            logger.info(f"Запрос пользователя: {query}")
            answer = self.agent.run(query)
            logger.info(f"Ответ агента: {answer}")
            return answer
        except Exception as e:
            logger.error(f"Ошибка выполнения агента: {e}")
            return f"Произошла ошибка: {str(e)}"
